In [1]:
import numpy as np
import pandas as pd
from random import randint

from sklearn.model_selection import train_test_split


In [9]:
ratings = {
    "col1": [randint(0,5) for i in range(10)],
    "col2": [randint(0,5) for i in range(10)],
    "col3": [randint(0,5) for i in range(10)],
    "col4": [randint(0,5) for i in range(10)],
    "col5": [randint(0,5) for i in range(10)],
    "col6": [randint(0,5) for i in range(10)],
    "col7": [randint(0,5) for i in range(10)],
    "col8": [randint(0,5) for i in range(10)],
}

In [10]:
df = pd.DataFrame(ratings)
df

,col1,col2,col3,col4,col5,col6,col7,col8
0,1,4,3,2,4,5,0,2
1,2,5,2,5,2,1,5,5
2,5,4,2,3,0,5,4,2
3,0,5,0,1,4,4,3,3
4,5,3,2,3,1,5,2,2
5,4,1,0,0,1,0,4,0
6,0,4,5,2,0,1,3,1
7,3,3,1,2,3,4,3,3
8,3,5,2,5,2,1,0,3
9,1,5,0,1,0,3,0,5


In [11]:
np.random.seed(111)
mask = np.random.choice([True, False], size=df.shape, p=[0.15, 0.85])
df = df.mask(mask)    

In [12]:
df

,col1,col2,col3,col4,col5,col6,col7,col8
0,1.0,4.0,3.0,2.0,4.0,NaN,NaN,2.0
1,2.0,5.0,2.0,5.0,NaN,1.0,5.0,5.0
2,5.0,NaN,NaN,3.0,0.0,5.0,4.0,2.0
3,0.0,5.0,0.0,NaN,4.0,NaN,3.0,3.0
4,5.0,3.0,2.0,NaN,1.0,5.0,2.0,2.0
5,4.0,1.0,0.0,0.0,1.0,0.0,NaN,0.0
6,0.0,4.0,5.0,2.0,0.0,1.0,NaN,NaN
7,NaN,NaN,NaN,2.0,3.0,4.0,3.0,NaN
8,3.0,5.0,2.0,5.0,2.0,1.0,0.0,3.0
9,NaN,5.0,0.0,1.0,0.0,3.0,0.0,5.0


In [13]:
train, test = train_test_split(df, test_size=0.2)

In [15]:
train


,col1,col2,col3,col4,col5,col6,col7,col8
0,1.0,4.0,3.0,2.0,4.0,NaN,NaN,2.0
7,NaN,NaN,NaN,2.0,3.0,4.0,3.0,NaN
4,5.0,3.0,2.0,NaN,1.0,5.0,2.0,2.0
1,2.0,5.0,2.0,5.0,NaN,1.0,5.0,5.0
3,0.0,5.0,0.0,NaN,4.0,NaN,3.0,3.0
2,5.0,NaN,NaN,3.0,0.0,5.0,4.0,2.0
5,4.0,1.0,0.0,0.0,1.0,0.0,NaN,0.0
6,0.0,4.0,5.0,2.0,0.0,1.0,NaN,NaN


In [18]:
df.mean(0)

col1    2.500000
col2    4.000000
col3    1.750000
col4    2.500000
col5    1.666667
col6    2.500000
col7    2.428571
col8    2.750000
dtype: float64